In [97]:
# Compiled from the previous works :
# https://github.com/genzellab/HM_RAT/blob/main/SYNCHRONIZATION/synch_editedbyOzge.py
# https://github.com/genzellab/HM_RAT/blob/main/SYNCHRONIZATION/Exctract_LEDs_28_01_2023.ipynb
# https://github.com/genzellab/HM_RAT/blob/main/SYNCHRONIZATION/synchronization.py

# Author: Param Rajpura
# 28th May 2023


%matplotlib notebook
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm, tnrange
from sklearn.decomposition import FastICA
from sklearn.cluster import KMeans
import pandas as pd
from scipy.signal import find_peaks,peak_prominences
from datetime import datetime , time , timedelta
import re

# Extract file paths from the user defined basepath 

In [98]:
 '''
 The basepath must contain the following files:
 1. Eye video files: .mp4 formats (12 files for each eye)
 2. X,y co-ordinates of crops for LED positions : .led_crop format (1 file containing 12 xy co-ordinates)
 3. Time stamp files containing framewise clock timestamps after linear regression: .csv format (12 files)
 4. Time stamps recorded from LED controller referred to as DIO: .dat format (3 files for red,blue and 
 initial systime)
 '''


# Reads all the mp4 files in the folder, checks for a led_crop_coordinates file and meta file
def get_video_files_with_metadata(basepath,led_xy=True,time_stamp=True):
    path = Path(basepath).resolve()
    videos_filepath_list = list(sorted(path.glob('*.mp4')))
#     print(videos_filepath_list)
    
    crop_xy_dict = {}
    # Verify if led_coordinates supplied
    if led_xy:
        crop_file_list = list(sorted(path.glob('*.led_crop')))
#         print(crop_file_list)
        if crop_file_list:
            # read crops coords for each video and store
            with open(crop_file_list[0]) as f:
                crop_txt = f.readlines()
#                 print(crop_txt)
            for line in tqdm(crop_txt):
                try:
                    vid_path, x, y = line.split(',')
                    crop_xy_dict[vid_path] = (int(x), int(y))
                except ValueError:
                    print("Faulty line:", line, 'Maybe led coordinates are missing?')
                    break
        else:
            raise Exception("File containing led crop coordinates not found.")
    if time_stamp:
        tsdata_filepath_list = list(sorted(path.glob('*.csv')))
        
    #TODO: Verify for single file path in the list to avoid conflicting data
    dio_file_path_dict={}
    dio_file_path_dict['init'] = list(sorted(path.glob('*maze.*.dat')))
    
    dio_file_path_dict['blue'] = list(sorted(path.glob('*maze_merged*Din1.dat')))
    dio_file_path_dict['red'] = list(sorted(path.glob('*maze_merged*Din2.dat')))
    return videos_filepath_list,crop_xy_dict,tsdata_filepath_list,dio_file_path_dict


# Functions to extract LED signals from video data 

In [99]:
def process_red_ica_signal(demixed, mix_weights,time_meta):
    fps = 30.0
    eD = 0.5       # expected Duty cycle of 0.5
    ef_red = 0.5   # expected frequency of 0.5 Hz
    ef_blue = 2.5  # expected frequency of 2.5 Hz
    
    dD = np.zeros(demixed.shape[1])
    df_red = np.zeros(demixed.shape[1])
    df_blue = np.zeros(demixed.shape[1])
    
    colors = {0: 'red', 1: 'blue', None: 'gray'}
    N = -1
    N_ICA = -1  # numbers of samples to use for ICA, -1 for all
    
    for n in range(demixed.shape[1]):

        # Check the mixing weights if the demixed signal polarity is reversed
        # (negative weights for ROI. Assuming rest of pixel array has weight zero, mean weight tells us sign.)
        flip_ica = mix_weights[n] < 0
        if flip_ica:
            demixed[:, n] = -demixed[:, n]

        km = KMeans(n_clusters=2, random_state=0).fit(demixed[:, n].reshape(-1, 1))
        y_km = km.predict(demixed[:, n].reshape(-1, 1))

        # check polarity, if necessary flip to match pulse polarity
        # print(f'Centers: {float(km.cluster_centers_[0]*1000):.2f}, {float(km.cluster_centers_[1]*1000):.2f}')
        centers = km.cluster_centers_.ravel()

        flip_kmeans = centers[0] > centers[1]
        flip = flip_ica ^ flip_kmeans
        # print(f'Polarity FLIP: {flip} (ICA {flip_ica}, kmeans {flip_kmeans})')
        if flip_kmeans:
            # print('Flipping!')
            y_km = np.abs(y_km-1)

        duty_cycle = y_km.sum()/len(y_km)
        freq = (np.diff(y_km)>0).sum()/len(y_km) * fps
        dD[n] = abs(eD-duty_cycle)
        df_red[n] = abs(ef_red - freq)
        df_blue[n] = abs(ef_blue - freq)

        # Attempt to identify the ICA signal as a color LED
        good_DC = dD[n] < 0.2 * eD
        good_freq = np.array([df_red[n] < ef_red * 0.1, df_blue[n] < ef_blue * 0.1])
        is_signal = good_DC and good_freq.sum()
        signal_color = good_freq.argmax() if is_signal else None

        sig_col = colors[signal_color]
        sig_name = 'None' if signal_color is None else colors[signal_color]

        if sig_col=='red':
            a = y_km[:N]
            df_temp = pd.DataFrame({'key' : [], "Red_LED_Intensity" : []})
            # "Red_LED_Intensity_%s" %(eye)
            df_temp['key'] = time_meta[0:(len(demixed[:N, n]-1))]
            df_temp["Red_LED_Intensity"] = demixed[:N, n]
    return df_temp

In [114]:
def process_video_with_metadata(file_path,xy_coord,ts_file_path,dio_file_path_dict):
    cap = cv2.VideoCapture(str(file_path))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # extract time stamps from the csv files based on sync_edited: as they are corrected timestamps
    
#     data = np.genfromtxt(ts_file_path, delimiter=',', names=True)
#     print(data)
#     time_meta = data['Timestamps_M']
    
    #TODO: optimise by reading selected columns
    df = pd.read_csv(str(ts_file_path), sep=',',parse_dates=['Timestamps_M'])#dtype=str)
    df['extracted_seconds_timestamp'] = pd.to_datetime(df['Timestamps_M'], unit='s')
#     print(df['extracted_seconds_timestamp']) # time_meta
    
    # read time stamps from the meta file : alternative but uncorrected
    
#     df = pd.read_csv(str(ts_file_path), sep=',',parse_dates=['callback_clock_ts'])#dtype=str)
#     df['extracted_seconds_timestamp'] = pd.to_datetime(df['callback_clock_ts'], unit='s')
    
    assert frame_count == len(df['extracted_seconds_timestamp']),"Frame count from video and metadata dont match"
    rgb_frames = np.empty((0,16,16,3))
#     while(cap.isOpened()):
    for i in range(1000):
        ret, frame = cap.read()
        if frame is None:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Start coordinate, here (5, 5)
        # represents the top left corner of rectangle
        start_point = (xy_dict[str(vfl[0])][0]-8, xy_dict[str(vfl[0])][1]-8)


        frame = frame[start_point[1]:start_point[1]+16,start_point[0]:start_point[0]+16]
        rgb_frames = np.append(rgb_frames,frame.reshape(-1, 16, 16, 3), axis=0)
#     print(rgb_frames)

    # number of components to extract from image crops: blue, red and noise
    nc = 3 
    ica = FastICA(n_components=nc, random_state=0)
    # reshape rgb_frames to a 2Darray
    X = rgb_frames.reshape(rgb_frames.shape[0], -1).astype(float) 
#     print(X.shape)
    # extraction of the independent signals 
    demixed = ica.fit_transform(X)
    mix_weights = ica.mixing_.mean(axis=0)
    
    #TODO: If need to check with blue, get blue_ica_df from the function
    red_ica_df = process_red_ica_signal(demixed,mix_weights,df['extracted_seconds_timestamp'])
    
    return red_ica_df


# Code to visualise the red_ica_df
#     fig, ax = plt.subplots(1, figsize=(40, 8)) #sharex="col", sharey=True )
#     ax.plot(red_ica_df['key'], red_ica_df['Red_LED_Intensity'], c='r')
#     ax.set_xlabel('Time')
#     ax.set_ylabel('Sum of ICAs (Red LED Intensities) of All Eyes')
#     # ax.set_xlim([df_final['key'][0], df_final['key'][width]])
#     plt.tight_layout()


In [115]:
def merge_ica_and_extract_com(eye_ica_list):
    # merge all eye data
    agg_ica = eye_ica_list
    # threhold
    agg_ica_thresh = agg_ica.Red_LED_Intensity > 0
    
    # Save binarized and summed red ICA and correspnding timstamps
    agg_ica_out = pd.DataFrame({'Time_in_seconds' : [], 'ICA_red' : []})
    agg_ica_out.Time_in_seconds = agg_ica['key']
    agg_ica_out.ICA_red = agg_ica_thresh.astype(int)
    
#     time_ica = ica_red['Time_in_seconds']
#     ica_int = ica_red['ICA_red']
    red_med = np.array(np.diff(agg_ica_out.ICA_red))
    red_med = np.append(0, red_med) # why add this 0 ? depends on any condition
    rising_edge_red = np.asarray(np.where(red_med==1)).flatten()
    falling_edge_red = np.asarray(np.where(red_med==-1)).flatten()
    com_ica = pd.DataFrame({'Center_of_mass' : []})  
    if agg_ica_out.Time_in_seconds[rising_edge_red[0]] < agg_ica_out.Time_in_seconds[falling_edge_red[0]]:  
        for i in range(min(len(rising_edge_red), len(falling_edge_red))):
            com_ica.at[i, 'Center_of_mass'] = agg_ica_out.Time_in_seconds[rising_edge_red[i]]
            +(agg_ica_out.Time_in_seconds[falling_edge_red[i]]
              -agg_ica_out.Time_in_seconds[rising_edge_red[i]])/2
    else:
        for i in range(min(len(rising_edge_red), len(falling_edge_red))):
            com_ica.at[i, 'Center_of_mass'] = agg_ica_out.Time_in_seconds[rising_edge_red[i]]
            +(agg_ica_out.Time_in_seconds[falling_edge_red[i+1]]
              -agg_ica_out.Time_in_seconds[rising_edge_red[i]])/2
    
    return com_ica

# Functions to extract DIO signals and centre of mass from metadata files 

In [116]:
#Extract DIOS

def readTrodesExtractedDataFile(filename):
    with open(filename, 'rb') as f:
        # Check if first line is start of settings block
        if f.readline().decode('ascii').strip() != '<Start settings>':
            raise Exception("Settings format not supported")
        fields = True
        fieldsText = {}
        for line in f:
            # Read through block of settings
            if(fields):
                line = line.decode('ascii').strip()
                # filling in fields dict
                if line != '<End settings>':
                    vals = line.split(': ')
                    fieldsText.update({vals[0].lower(): vals[1]})
                # End of settings block, signal end of fields
                else:
                    fields = False
                    dt = parseFields(fieldsText['fields'])
                    fieldsText['data'] = np.zeros([1], dtype = dt)
                    break
        # Reads rest of file at once, using dtype format generated by parseFields()
        dt = parseFields(fieldsText['fields'])
        data = np.fromfile(f, dt)
        fieldsText.update({'data': data})
        return fieldsText
# Parses last fields parameter (<time uint32><...>) as a single string
# Assumes it is formatted as <name number * type> or <name type>
# Returns: np.dtype
def parseFields(fieldstr):
    # Returns np.dtype from field string
    sep = re.split('\s', re.sub(r"\>\<|\>|\<", ' ', fieldstr).strip())
    # print(sep)
    typearr = []
    # Every two elmts is fieldname followed by datatype
    for i in range(0,sep.__len__(), 2):
        fieldname = sep[i]
        repeats = 1
        ftype = 'uint32'
        # Finds if a <num>* is included in datatype
        if sep[i+1].__contains__('*'):
            temptypes = re.split('\*', sep[i+1])
            # Results in the correct assignment, whether str is num*dtype or dtype*num
            ftype = temptypes[temptypes[0].isdigit()]
            repeats = int(temptypes[temptypes[1].isdigit()])
        else:
            ftype = sep[i+1]
        try:
            fieldtype = getattr(np, ftype)
        except AttributeError:
            print(ftype + " is not a valid field type.\n")
            exit(1)
        else:
            typearr.append((str(fieldname), fieldtype, repeats))
    return np.dtype(typearr)


def extract_dio_com(dio_file_path_dict):
    sys_time_dict = readTrodesExtractedDataFile(dio_file_path_dict['init'][0])
    sys_time = int(sys_time_dict['system_time_at_creation'])/1000
    timestamp_at_creation = int(sys_time_dict['timestamp_at_creation'])#/1000
    sys_time_dt = datetime.utcfromtimestamp(sys_time)
    print(pd.to_datetime(sys_time, unit='s'),sys_time_dt,datetime.utcfromtimestamp(timestamp_at_creation/1000))
    
    red_dict_dio = readTrodesExtractedDataFile(dio_file_path_dict['red'][0])
    red_DIO = red_dict_dio['data']
    
    red_DIO_ts = [((sys_time_dt + timedelta(seconds = (i[0]-timestamp_at_creation)/ 30000)).timestamp(),
                   i[1]) for i in red_DIO]
    print(red_DIO)
    red_DIO_df  = pd.DataFrame({"Time_Stamp_(DIO)" : [datetime.fromtimestamp(i[0]) for i in red_DIO_ts], 
                                "Time_in_seconds_(DIO)" : [str(i[0]) for i in red_DIO_ts], 
                                "State": [i[1] for i in red_DIO_ts]} )
#     print(red_DIO_ts)
#     print(red_DIO_df)
    
    blue_dict_dio = readTrodesExtractedDataFile(dio_file_path_dict['blue'][0])
    blue_DIO = blue_dict_dio['data']
    blue_DIO_ts = [((sys_time_dt + timedelta(seconds = (i[0]-timestamp_at_creation)/ 30000)).timestamp() , 
                    i[1]) for i in blue_DIO]
    blue_DIO_df  = pd.DataFrame({"Time_Stamp_(DIO)" : [datetime.fromtimestamp(i[0]) for i in blue_DIO_ts], 
                                 "Time_in_seconds_(DIO)" : [str(i[0]) for i in blue_DIO_ts], 
                                 "State": [i[1] for i in blue_DIO_ts]} )
    
    
    com_dio = pd.DataFrame({'Center_of_mass' : []})
    if red_DIO_df["State"][0]==1:
        for i in range(2, len(red_DIO_df["State"]), 2):
            com_dio.at[(i-2)/2, 'Center_of_mass'] = red_DIO_df["Time_Stamp_(DIO)"][i-2]
            +(red_DIO_df["Time_Stamp_(DIO)"][i]-red_DIO_df["Time_Stamp_(DIO)"][i-2])/2
    else:
        for i in range(3, len(red_DIO_df["State"]), 2):
            com_dio.at[(((i-1)/2)-1), 'Center_of_mass'] = red_DIO_df["Time_Stamp_(DIO)"][i-2]
            +(red_DIO_df["Time_Stamp_(DIO)"][i]-red_DIO_df["Time_Stamp_(DIO)"][i-2])/2
    return com_dio

# Main code using previously defined functions to generate ICA vs DIO visualisation and sync

In [117]:

# Get file list paths and the metadata paths related to it : dio timestamps, xy co-ords, ???
vfl,xy_dict,tsfl,dio_file_path_dict = get_video_files_with_metadata('./maze_videos/')
print(xy_dict[str(vfl[0])])

# TODO: loop over each video file to get the df
eye_ica_out = process_video_with_metadata(vfl[0],xy_dict[str(vfl[0])],tsfl[0],dio_file_path_dict)



# process the combined ica signals and get centre of mass for the aggregated signal from all eyes
ica_com = merge_ica_and_extract_com(eye_ica_out)

# extract dio signal, time stamps, 
# process the dio signals and timestamps, and 
# get centre of mass for dio signals
dio_com = extract_dio_com(dio_file_path_dict)

print(ica_com)


print(dio_com)

  0%|          | 0/12 [00:00<?, ?it/s]

(566, 731)


/tmp/ipykernel_5759/2445062803.py:57: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(typearr)


2020-11-09 11:33:59.364999936 2020-11-09 11:33:59.365000 1970-01-01 00:23:36.575000
[(  2056320, 1) (  2065607, 0) (  2095643, 1) ... (119008823, 1)
 (119038861, 0) (119068929, 1)]
                   Center_of_mass
0   1970-01-19 13:48:38.059949640
1   1970-01-19 13:48:38.061949280
2   1970-01-19 13:48:38.063948920
3   1970-01-19 13:48:38.065948550
4   1970-01-19 13:48:38.067948190
5   1970-01-19 13:48:38.069947830
6   1970-01-19 13:48:38.071947470
7   1970-01-19 13:48:38.073980430
8   1970-01-19 13:48:38.075980070
9   1970-01-19 13:48:38.077979710
10  1970-01-19 13:48:38.079979340
11  1970-01-19 13:48:38.081978980
12  1970-01-19 13:48:38.083978620
13  1970-01-19 13:48:38.085978260
14  1970-01-19 13:48:38.087977890
15  1970-01-19 13:48:38.090010860
                  Center_of_mass
0     2020-11-09 11:34:20.689833
1     2020-11-09 11:34:22.000600
2     2020-11-09 11:34:24.004133
3     2020-11-09 11:34:26.007633
4     2020-11-09 11:34:28.011167
...                          ...
1942  2020